In [ ]:
pip install google-adk


In [ ]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

1.3: Import ADK components¶
Now, import the specific components you'll need from the Agent Development Kit and the Generative AI library. This keeps your code organized and ensures we have access to the necessary building blocks.

In [ ]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

1.4: Configure Retry Options¶
When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. Retry options automatically handle these failures by retrying the request with exponential backoff.

In [ ]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

User → InitialSearchAgent → (Status, URL)
    ├── If NOT FOUND → SupplementalSearchAgent → return URL
    └── If FOUND → FileSearchAgent → “Extracted Knowledge”
            ↓
          GapFinderAgent → missing items?
            ├── If YES → SupplementalSearchAgent → fill gaps → back to GapFinderAgent
            └── If NO → proceed
                  ↓
        ReportGeneratorAgent (CodeExecution Tool → PDF/PPT)

Agent 1 — InitialSearchAgent

Search Google for “[company] annual report last year".

In [ ]:
InitialSearchAgent = Agent(
    name="InitialSearchAgent",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    instruction="""
You search the web to locate the company's latest annual report.
Return json exactly:
status: FOUND or NOT_FOUND
url: direct link to the PDF if found
company: {company_name}
""",
    output_key="search_result"
    description="Extracts and structures data from company and person research into a JSON object.",
    output_schema= 
)
    

In [ ]:
# test this agent etisalat 

In [ ]:
    output_key="research_findings",  # The result of this agent will be stored in the session state with this key.
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),